In [1]:
import os
import pandas as pd
import numpy as np
import random
import torch
from transformers import AutoTokenizer, AutoModel


In [2]:
def get_sequences(filename):
    with open(filename) as f:
        lines = [line.rstrip('\n') for line in f]
    return lines

def create_sequences(folder_path):
    filenames = []
    for filename in os.listdir(folder_path):
        if '.txt' in filename:
            f = "../../data/" + filename
            filenames.append(f)
    lines = []
    for file in filenames:
        line = get_sequences(file)
        lines = lines + line
    return lines

In [3]:
def create_csv(folder_path, data):
    l = len(data)
    labels = [random.randint(0, 1) for _ in range(l)]
    num_train = int(l * 0.8)
    num_test = ((l - num_train) // 2) + num_train
    
    train_dict = {'sequence': data[:num_train], 'label': labels[:num_train]}
    test_dict ={'sequence': data[num_train:num_test], 'label': labels[num_train:num_test]}
    dev_dict = {'sequence': data[num_test:], 'label': labels[num_test:]}
    
    train_df = pd.DataFrame(train_dict)
    test_df = pd.DataFrame(test_dict)
    dev_df = pd.DataFrame(dev_dict)

    train_path = folder_path + '/train.csv'
    test_path = folder_path + '/test.csv'
    dev_path = folder_path + '/dev.csv'
    train_df.to_csv(train_path, index=False)
    test_df.to_csv(test_path, index=False)
    dev_df.to_csv(dev_path, index=False)
    return 'csv files created'

In [4]:
#this creates csv files, only run once if you don't have files
#k = create_sequences(folder_path = "../../data")
#create_csv("../../data/finetuning_data", k[:200])

In [5]:
from transformers.models.bert.configuration_bert import BertConfig

config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M")
model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, config=config)

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at zhihan1996/DNABERT-2-117M were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializin

In [6]:
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(4096, 768, padding_idx=0)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertUnpadAttention(
          (self): BertUnpadSelfAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (mlp): BertGatedLinearUnitMLP(
          (gated_layers): Linear(in_features=768, out_features=6144, bias=False)
          (act): GELU(approximate='none')
          

In [9]:
print(device)

cuda:0


In [10]:
dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"]
hidden_states = model(inputs)[0] # [1, sequence_length, 768]

# embedding with mean pooling
embedding_mean = torch.mean(hidden_states[0], dim=0)
print(embedding_mean.shape) # expect to be 768

# embedding with max pooling
embedding_max = torch.max(hidden_states[0], dim=0)[0]
print(embedding_max.shape) # expect to be 768

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)